In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import dask.dataframe as dd

In [2]:
# Load datasets
wa_tax_exemptions = pd.read_csv("/Users/vuritysamuel/Documents/Regression Project/Data/WA Tax Exemptions Eligibility.csv")
wa_employment = pd.read_csv("/Users/vuritysamuel/Documents/Regression Project/Data/WA Employment.csv")
re_production_consumption = pd.read_excel("/Users/vuritysamuel/Documents/Regression Project/Data/RE Production & Consumption by Source.xlsx")
re_consumption_industrial = pd.read_excel("/Users/vuritysamuel/Documents/Regression Project/Data/RE Consumption - Industrial.xlsx")
primary_energy_consumption = pd.read_excel("/Users/vuritysamuel/Documents/Regression Project/Data/Primary Energy Consumption by Source.xlsx")
per_capita_income = pd.read_excel("/Users/vuritysamuel/Documents/Regression Project/Data/Per Capita Personal Income by County.xlsx")
ev_title_registration = pd.read_csv("/Users/vuritysamuel/Documents/Regression Project/Data/EV Title & Registration.csv")
ev_population = pd.read_csv("/Users/vuritysamuel/Documents/Regression Project/Data/EV Population by County.csv")

# List of column names
column_names = {
    "WA Tax Exemptions Eligibility": wa_tax_exemptions.columns.tolist(),
    "WA Employment": wa_employment.columns.tolist(),
    "RE Production & Consumption by Source": re_production_consumption.columns.tolist(),
    "RE Consumption - Industrial": re_consumption_industrial.columns.tolist(),
    "Primary Energy Consumption by Source": primary_energy_consumption.columns.tolist(),
    "Per Capita Personal Income by County": per_capita_income.columns.tolist(),
    "EV Title & Registration": ev_title_registration.columns.tolist(),
    "EV Population by County": ev_population.columns.tolist(),

}
wa_employment.columns = wa_employment.columns.str.replace('Washington!!', '', regex=False)
# Display the column names
for dataset, columns in column_names.items():
    print(f"{dataset}:\n{columns}\n")

WA Tax Exemptions Eligibility:
['Model Year', 'Make', 'Vehicle Model Description', 'Alternative Fuel Type']

WA Employment:
['Label (Grouping)', 'Washington!!Estimate', 'Washington!!Margin of Error', 'Washington!!Percent', 'Washington!!Percent Margin of Error']

RE Production & Consumption by Source:
['Month', 'Wood Energy Production', 'Biofuels Production', 'Total Biomass Energy Production', 'Total Renewable Energy Production', 'Hydroelectric Power Consumption', 'Geothermal Energy Consumption', 'Solar Energy Consumption', 'Wind Energy Consumption', 'Wood Energy Consumption', 'Waste Energy Consumption', 'Biofuels Consumption', 'Total Biomass Energy Consumption', 'Total Renewable Energy Consumption']

RE Consumption - Industrial:
['Month', 'Hydroelectric Power Consumed by the Industrial Sector', 'Geothermal Energy Consumed by the Industrial Sector', 'Solar Energy Consumed by the Industrial Sector', 'Wind Energy Consumed by the Industrial Sector', 'Wood Energy Consumed by the Industrial 

In [3]:
# Drop columns from WA Employment
wa_employment = wa_employment.drop(['Margin of Error', 'Percent Margin of Error'], axis=1, errors='ignore')

# Drop columns from RE Production & Consumption by Source
re_production_consumption = re_production_consumption.drop(['Month', 'Wood Energy Production', 'Waste Energy Consumption'], axis=1, errors='ignore')

# Drop columns from RE Consumption - Industrial
re_consumption_industrial = re_consumption_industrial.drop(['Month', 
    'Wood Energy Consumed by the Industrial Sector', 
    'Waste Energy Consumed by the Industrial Sector'], axis=1, errors='ignore')

# Drop columns from Primary Energy Consumption by Source
primary_energy_consumption = primary_energy_consumption.drop(['Month', 
    'Coal Consumption', 
    'Petroleum Consumption (Excluding Biofuels)'], axis=1, errors='ignore')

# Drop columns from Per Capita Personal Income by County
per_capita_income = per_capita_income.drop(['Per Capita Personal Income Rank in State', 
    'Percent Change Rank in State'], axis=1, errors='ignore')

# Drop columns from EV Title & Registration
ev_title_registration = ev_title_registration.drop(['VIN (1-10)', 
    'DOL Vehicle ID', 
    'Odometer Reading', 
    'Odometer Reading Description', 
    'Base MSRP', 
    'Sale Price'], axis=1, errors='ignore')


In [4]:

# Function to check for missing values
def check_missing_values(df, name):
    missing = df.isnull().sum()
    print(f"\nMissing values in {name}:\n{missing[missing > 0]}")

# Check for missing values in all datasets
datasets = {
    "WA Tax Exemptions": wa_tax_exemptions,
    "WA Employment": wa_employment,
    "RE Production & Consumption": re_production_consumption,
    "RE Consumption - Industrial": re_consumption_industrial,
    "Primary Energy Consumption": primary_energy_consumption,
    "Per Capita Personal Income": per_capita_income,
    "EV Title & Registration": ev_title_registration,
    "EV Population": ev_population
}

for name, df in datasets.items():
    check_missing_values(df, name)
    
wa_employment = wa_employment[~wa_employment['Label (Grouping)'].isin(['EMPLOYMENT STATUS', 'COMMUTING TO WORK', 'OCCUPATION', 'INDUSTRY','CLASS OF WORKER','INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)','HEALTH INSURANCE COVERAGE','PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL',''])]
ev_title_registration.dropna(subset=['County', 'City', 'State', 'Postal Code'], inplace=True)
median_electric_range = ev_title_registration['Electric Range'].median()
ev_title_registration['Electric Range'].fillna(median_electric_range, inplace=True)
ev_title_registration.drop(columns=['Sale Date'], inplace=True)
ev_title_registration.drop(columns=['Transportation Electrification Fee Paid', 'Hybrid Vehicle Electrification Fee Paid'], inplace=True)
ev_title_registration.drop(columns=['Legislative District'], inplace=True)
ev_population.dropna(subset=['County', 'State'], inplace=True)
ev_title_registration = ev_title_registration.drop(columns=['Sale Date'])
ev_title_registration['Transportation Electrification Fee Paid'].fillna("Not Paid", inplace=True)
ev_title_registration['Hybrid Vehicle Electrification Fee Paid'].fillna("Not Paid", inplace=True)
ev_title_registration = ev_title_registration.drop(columns=['Legislative District'])


Missing values in WA Tax Exemptions:
Series([], dtype: int64)

Missing values in WA Employment:
Estimate    8
Percent     8
dtype: int64

Missing values in RE Production & Consumption:
Series([], dtype: int64)

Missing values in RE Consumption - Industrial:
Series([], dtype: int64)

Missing values in Primary Energy Consumption:
Series([], dtype: int64)

Missing values in Per Capita Personal Income:
Series([], dtype: int64)

Missing values in EV Title & Registration:
Electric Range                                 11
Sale Date                                  842439
County                                         61
City                                          108
State                                           1
Postal Code                                    83
Transportation Electrification Fee Paid     91038
Hybrid Vehicle Electrification Fee Paid     91038
2020 GEOID                                     61
Legislative District                         4133
Electric Utility            

KeyError: "['Sale Date'] not found in axis"

In [6]:
# Function to display data types
def display_data_types(df, name):
    print(f"\nData types in {name}:\n{df.dtypes}")

# Display data types for all datasets
datasets = {
    "WA Tax Exemptions": wa_tax_exemptions,
    "WA Employment": wa_employment,
    "RE Production & Consumption": re_production_consumption,
    "RE Consumption - Industrial": re_consumption_industrial,
    "Primary Energy Consumption": primary_energy_consumption,
    "Per Capita Personal Income": per_capita_income,
    "EV Title & Registration": ev_title_registration,
    "EV Population": ev_population
}

for name, df in datasets.items():
    display_data_types(df, name)


Data types in WA Tax Exemptions:
Model Year                    int64
Make                         object
Vehicle Model Description    object
Alternative Fuel Type        object
dtype: object

Data types in WA Employment:
Label (Grouping)    object
Estimate            object
Percent             object
dtype: object

Data types in RE Production & Consumption:
Biofuels Production                    object
Total Biomass Energy Production       float64
Total Renewable Energy Production     float64
Hydroelectric Power Consumption       float64
Geothermal Energy Consumption         float64
Solar Energy Consumption               object
Wind Energy Consumption                object
Wood Energy Consumption               float64
Biofuels Consumption                   object
Total Biomass Energy Consumption      float64
Total Renewable Energy Consumption    float64
dtype: object

Data types in RE Consumption - Industrial:
Hydroelectric Power Consumed by the Industrial Sector                    fl

In [7]:
# Convert columns to numeric types in the WA Employment dataset
wa_employment['Estimate'] = pd.to_numeric(wa_employment['Estimate'], errors='coerce')
wa_employment['Percent'] = pd.to_numeric(wa_employment['Percent'], errors='coerce')

# Convert columns to numeric types in the RE Production & Consumption dataset
re_production_consumption['Biofuels Production'] = pd.to_numeric(re_production_consumption['Biofuels Production'], errors='coerce')
re_production_consumption['Solar Energy Consumption'] = pd.to_numeric(re_production_consumption['Solar Energy Consumption'], errors='coerce')
re_production_consumption['Wind Energy Consumption'] = pd.to_numeric(re_production_consumption['Wind Energy Consumption'], errors='coerce')
re_production_consumption['Biofuels Consumption'] = pd.to_numeric(re_production_consumption['Biofuels Consumption'], errors='coerce')

# Convert columns to numeric types in the RE Consumption - Industrial dataset
re_consumption_industrial['Geothermal Energy Consumed by the Industrial Sector'] = pd.to_numeric(re_consumption_industrial['Geothermal Energy Consumed by the Industrial Sector'], errors='coerce')
re_consumption_industrial['Solar Energy Consumed by the Industrial Sector'] = pd.to_numeric(re_consumption_industrial['Solar Energy Consumed by the Industrial Sector'], errors='coerce')
re_consumption_industrial['Wind Energy Consumed by the Industrial Sector'] = pd.to_numeric(re_consumption_industrial['Wind Energy Consumed by the Industrial Sector'], errors='coerce')
re_consumption_industrial['Fuel Ethanol, Excluding Denaturant, Consumed by the Industrial Sector'] = pd.to_numeric(re_consumption_industrial['Fuel Ethanol, Excluding Denaturant, Consumed by the Industrial Sector'], errors='coerce')
re_consumption_industrial['Biomass Losses and Co-products in the Industrial Sector'] = pd.to_numeric(re_consumption_industrial['Biomass Losses and Co-products in the Industrial Sector'], errors='coerce')

# Convert columns to numeric types in the Primary Energy Consumption dataset
primary_energy_consumption['Solar Energy Consumption'] = pd.to_numeric(primary_energy_consumption['Solar Energy Consumption'], errors='coerce')
primary_energy_consumption['Wind Energy Consumption'] = pd.to_numeric(primary_energy_consumption['Wind Energy Consumption'], errors='coerce')

# Convert date columns to datetime types in the EV Title & Registration dataset
ev_title_registration['Transaction Date'] = pd.to_datetime(ev_title_registration['Transaction Date'], errors='coerce')

# Convert date columns to datetime types in the EV Population dataset
ev_population['Date'] = pd.to_datetime(ev_population['Date'], errors='coerce')

for name, df in datasets.items():
    display_data_types(df, name)


Data types in WA Tax Exemptions:
Model Year                    int64
Make                         object
Vehicle Model Description    object
Alternative Fuel Type        object
dtype: object

Data types in WA Employment:
Label (Grouping)     object
Estimate            float64
Percent             float64
dtype: object

Data types in RE Production & Consumption:
Biofuels Production                   float64
Total Biomass Energy Production       float64
Total Renewable Energy Production     float64
Hydroelectric Power Consumption       float64
Geothermal Energy Consumption         float64
Solar Energy Consumption              float64
Wind Energy Consumption               float64
Wood Energy Consumption               float64
Biofuels Consumption                  float64
Total Biomass Energy Consumption      float64
Total Renewable Energy Consumption    float64
dtype: object

Data types in RE Consumption - Industrial:
Hydroelectric Power Consumed by the Industrial Sector                   

In [8]:
# Define file paths for each cleaned dataset
file_paths = {
    "WA_Tax_Exemptions_Cleaned.csv": wa_tax_exemptions,
    "WA_Employment_Cleaned.csv": wa_employment,
    "RE_Production_Consumption_Cleaned.csv": re_production_consumption,
    "RE_Consumption_Industrial_Cleaned.csv": re_consumption_industrial,
    "Primary_Energy_Consumption_Cleaned.csv": primary_energy_consumption,
    "Per_Capita_Income_Cleaned.csv": per_capita_income,
    "EV_Title_Registration_Cleaned.csv": ev_title_registration,
    "EV_Population_Cleaned.csv": ev_population
}

# Loop through each DataFrame and save as CSV
for file_name, df in file_paths.items():
    df.to_csv(file_name, index=False)
    print(f"{file_name} has been saved.")

WA_Tax_Exemptions_Cleaned.csv has been saved.
WA_Employment_Cleaned.csv has been saved.
RE_Production_Consumption_Cleaned.csv has been saved.
RE_Consumption_Industrial_Cleaned.csv has been saved.
Primary_Energy_Consumption_Cleaned.csv has been saved.
Per_Capita_Income_Cleaned.csv has been saved.
EV_Title_Registration_Cleaned.csv has been saved.
EV_Population_Cleaned.csv has been saved.


In [9]:
base_path = "/Users/vuritysamuel/Documents/Regression Project/Data/"
ev_population_path = base_path + "EV_Population_Cleaned.csv"
per_capita_income_path = base_path + "Per_Capita_Income_Cleaned.csv"
ev_title_registration_path = base_path + "EV_Title_Registration_Cleaned.csv"  # Large file
wa_tax_exemptions_path = base_path + "WA_Tax_Exemptions_Cleaned.csv"

In [2]:
import dask.dataframe as dd

# Define paths
base_path = "/Users/vuritysamuel/Documents/Regression Project/Data/"
ev_population_path = base_path + "EV_Population_Cleaned.csv"
per_capita_income_path = base_path + "Per_Capita_Income_Cleaned.csv"
ev_title_registration_path = base_path + "EV_Title_Registration_Cleaned.csv"  # Large file

# Load and clean EV Population data
ev_population_dd = dd.read_csv(ev_population_path, usecols=[
    'Date', 'County', 'State', 'Vehicle Primary Use',
    'Battery Electric Vehicles (BEVs)',
    'Plug-In Hybrid Electric Vehicles (PHEVs)',
    'Electric Vehicle (EV) Total', 'Total Vehicles', 'Percent Electric Vehicles'
])
ev_population_dd = ev_population_dd[ev_population_dd['State'] == 'WA']
ev_population_dd['Date'] = dd.to_datetime(ev_population_dd['Date'], errors='coerce')
ev_population_dd['County'] = ev_population_dd['County'].str.lower().str.strip()
ev_population_dd = ev_population_dd.drop(columns=['State'])

# Load and clean Per Capita Income data
per_capita_income_dd = dd.read_csv(per_capita_income_path, usecols=[
    'County Name', 'Per Capita Personal Income 2020',
    'Per Capita Personal Income 2021', 'Per Capita Personal Income 2022',
    'Percent Change from preceding period 2020-2021',
    'Percent Change from preceding period 2021-2022'
])
per_capita_income_dd = per_capita_income_dd.rename(columns={'County Name': 'County'})
per_capita_income_dd['County'] = per_capita_income_dd['County'].str.lower().str.strip()

# Load and clean EV Title Registration data
ev_title_registration_dd = dd.read_csv(ev_title_registration_path, usecols=[
    'County', 'Model Year', 'Make', 'Electric Range', 'Transaction Date', 'State'
])
ev_title_registration_dd = ev_title_registration_dd[ev_title_registration_dd['State'] == 'WA']
ev_title_registration_dd['Transaction Date'] = dd.to_datetime(ev_title_registration_dd['Transaction Date'], errors='coerce')
ev_title_registration_dd['County'] = ev_title_registration_dd['County'].str.lower().str.strip()
ev_title_registration_dd = ev_title_registration_dd.drop(columns=['State'])

# Save cleaned versions of each dataset to Parquet for merging
ev_population_dd.to_parquet(base_path + "Cleaned_EV_Population.parquet")
per_capita_income_dd.to_parquet(base_path + "Cleaned_Per_Capita_Income.parquet")
ev_title_registration_dd.to_parquet(base_path + "Cleaned_EV_Title_Registration.parquet")

print("Selected datasets cleaned and saved in Parquet format.")

Selected datasets cleaned and saved in Parquet format.


In [3]:
# Load the main cleaned data files with Dask
ev_population_dd = dd.read_parquet(base_path + "Cleaned_EV_Population.parquet")
per_capita_income_dd = dd.read_parquet(base_path + "Cleaned_Per_Capita_Income.parquet")

# Step 1: Merge EV Population with Per Capita Income on 'County'
merged_ev_population_income_dd = ev_population_dd.merge(per_capita_income_dd, on='County')

# Persist the intermediate result to manage memory, then save to disk
merged_ev_population_income_dd = merged_ev_population_income_dd.persist()
merged_ev_population_income_dd.to_parquet(base_path + "merged_ev_population_income.parquet")

# Step 2: Load and filter EV Title Registration data for Washington state only
ev_title_registration_dd = dd.read_parquet(base_path + "Cleaned_EV_Title_Registration.parquet")

# Merge with the previously saved intermediate data
merged_ev_population_income_dd = dd.read_parquet(base_path + "merged_ev_population_income.parquet")
final_merged_data_dd = merged_ev_population_income_dd.merge(ev_title_registration_dd, on='County')

# Save the final merged data to Parquet
final_merged_data_dd.to_parquet(base_path + "final_merged_ev_data.parquet")

In [4]:
print("Columns and Types:\n", final_merged_data_dd.dtypes)

Columns and Types:
 Date                                              datetime64[ns]
County                                                    object
Vehicle Primary Use                                       object
Battery Electric Vehicles (BEVs)                           int64
Plug-In Hybrid Electric Vehicles (PHEVs)                   int64
Electric Vehicle (EV) Total                                int64
Total Vehicles                                             int64
Percent Electric Vehicles                                float64
Per Capita Personal Income 2020                            int64
Per Capita Personal Income 2021                            int64
Per Capita Personal Income 2022                            int64
Percent Change from preceding period 2020-2021           float64
Percent Change from preceding period 2021-2022           float64
Model Year                                                 int64
Make                                                      object
Elect

In [5]:
# Drop rows with any missing values
final_merged_data_dd = final_merged_data_dd.dropna()

# Save the cleaned data without missing rows back to Parquet (or continue with your analysis)
final_merged_data_dd.to_parquet(base_path + "final_merged_ev_data_no_missing.parquet")

print("Rows with missing values have been dropped, and the cleaned data is saved.")

Rows with missing values have been dropped, and the cleaned data is saved.


In [6]:
import dask.dataframe as dd

# Define paths
final_data_path = base_path + "final_merged_ev_data.parquet"
output_file_path = base_path + "final_merged_ev_data_no_duplicates.parquet"

# Load data with Dask
dask_df = dd.read_parquet(final_data_path)

# Drop duplicates across all columns
dask_df = dask_df.drop_duplicates()

# Save the unique rows back to Parquet
dask_df.to_parquet(output_file_path)

print("Data with unique rows saved to Parquet.")

Data with unique rows saved to Parquet.


In [7]:
import dask.dataframe as dd

# Define the path to the cleaned Parquet file
output_file_path = base_path + "final_merged_ev_data_no_duplicates.parquet"

# Load the cleaned data
final_cleaned_df = dd.read_parquet(output_file_path)

# Check the number of rows and columns
n_rows = final_cleaned_df.shape[0].compute()
n_columns = final_cleaned_df.shape[1]

print(f"Number of rows: {n_rows}")
print(f"Number of columns: {n_columns}")

# Display the first few rows as a sample
print("Sample Data:\n", final_cleaned_df.head())

Number of rows: 86737194
Number of columns: 17
Sample Data:
         Date   County Vehicle Primary Use  Battery Electric Vehicles (BEVs)  \
0 2017-04-30  lincoln               Truck                                 0   
1 2021-05-31  lincoln           Passenger                                 5   
2 2019-08-31  lincoln               Truck                                 0   
3 2021-04-30  lincoln           Passenger                                 5   
4 2021-02-28  lincoln               Truck                                 0   

   Plug-In Hybrid Electric Vehicles (PHEVs)  Electric Vehicle (EV) Total  \
0                                         0                            0   
1                                        13                           18   
2                                         0                            0   
3                                        13                           18   
4                                         0                            0   

   Tota

In [8]:
import dask.dataframe as dd

# Define the path to the final merged dataset
final_data_path = base_path + "final_merged_ev_data.parquet"

# Load the data with Dask
final_merged_data_dd = dd.read_parquet(final_data_path)

# Filter for data from 2019 to 2023
filtered_data_dd = final_merged_data_dd[
    (final_merged_data_dd['Date'] >= '2019-01-01') & (final_merged_data_dd['Date'] <= '2023-12-31')
]

# Save the filtered dataset to a new file
filtered_data_path = base_path + "filtered_ev_data_2019_2023.parquet"
filtered_data_dd.to_parquet(filtered_data_path)

print("Filtered data saved to:", filtered_data_path)

Filtered data saved to: /Users/vuritysamuel/Documents/Regression Project/Data/filtered_ev_data_2019_2023.parquet


In [9]:
# Load the filtered data from step 1
filtered_data_dd = dd.read_parquet(filtered_data_path)

# Filter for relevant vehicle types (BEVs and PHEVs)
filtered_ev_types_dd = filtered_data_dd[
    (filtered_data_dd['Battery Electric Vehicles (BEVs)'] > 0) |
    (filtered_data_dd['Plug-In Hybrid Electric Vehicles (PHEVs)'] > 0)
]

# Save the filtered dataset to a new file for further analysis
final_filtered_data_path = base_path + "filtered_ev_data_2019_2023_ev_types.parquet"
filtered_ev_types_dd.to_parquet(final_filtered_data_path)

print("Filtered data by EV types saved to:", final_filtered_data_path)

Filtered data by EV types saved to: /Users/vuritysamuel/Documents/Regression Project/Data/filtered_ev_data_2019_2023_ev_types.parquet


In [10]:
# Load the final filtered data
filtered_ev_types_dd = dd.read_parquet(final_filtered_data_path)

# Compute the number of rows and columns
num_rows = len(filtered_ev_types_dd)
num_columns = len(filtered_ev_types_dd.columns)

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Show the first few rows to verify the data
print("Sample data:")
print(filtered_ev_types_dd.head())

Number of rows: 125238489
Number of columns: 17
Sample data:
         Date   County Vehicle Primary Use  Battery Electric Vehicles (BEVs)  \
1  2021-05-31  lincoln           Passenger                                 5   
3  2021-04-30  lincoln           Passenger                                 5   
13 2021-07-31  lincoln           Passenger                                 6   
16 2020-11-30  lincoln           Passenger                                 5   
17 2023-10-31  lincoln           Passenger                                21   

    Plug-In Hybrid Electric Vehicles (PHEVs)  Electric Vehicle (EV) Total  \
1                                         13                           18   
3                                         13                           18   
13                                        15                           21   
16                                        11                           16   
17                                        24                           45

In [13]:
import dask.dataframe as dd

# Load data
final_data_path = base_path + "final_merged_ev_data.parquet"
data_dd = dd.read_parquet(final_data_path)

# Remove columns that aren’t central to understanding expansion potential
columns_to_keep = [
    'Date', 'County', 'Battery Electric Vehicles (BEVs)', 
    'Plug-In Hybrid Electric Vehicles (PHEVs)', 'Electric Vehicle (EV) Total',
    'Total Vehicles', 'Percent Electric Vehicles',
    'Per Capita Personal Income 2020', 'Per Capita Personal Income 2021', 
    'Per Capita Personal Income 2022', 
    'Percent Change from preceding period 2020-2021', 
    'Percent Change from preceding period 2021-2022'
]

filtered_data_dd = data_dd[columns_to_keep]

# Check number of rows and columns after column filtering
print("Number of rows after column filtering:", len(filtered_data_dd))
print("Number of columns after column filtering:", len(filtered_data_dd.columns))

# Save the filtered data to check the result before proceeding with row filtering or aggregation
filtered_data_dd.to_parquet(base_path + "filtered_ev_data_columns.parquet")

Number of rows after column filtering: 217322586
Number of columns after column filtering: 12


In [15]:
import dask.dataframe as dd

# Load the filtered data from the previous step
filtered_data_path = base_path + "filtered_ev_data_columns.parquet"
filtered_data_dd = dd.read_parquet(filtered_data_path)

# Convert Dask DataFrame to Pandas for sampling
filtered_data_pd = filtered_data_dd.compute()

# Take a random sample of 10,000 rows
sampled_data = filtered_data_pd.sample(n=10000, random_state=42)

# Check number of rows and columns in the sample
print(f"Number of rows in the sample: {sampled_data.shape[0]}")
print(f"Number of columns in the sample: {sampled_data.shape[1]}")

# Optionally, save the sample to a new Parquet file
sampled_data.to_parquet(base_path + "sampled_ev_data_10000.parquet", index=False)

# View a sample of the data to verify
print("Sampled Data:\n", sampled_data.head())

Number of rows in the sample: 10000
Number of columns in the sample: 12
Sampled Data:
                Date   County  Battery Electric Vehicles (BEVs)  \
13240033 2018-10-31     king                                 7   
5174968  2019-01-31    clark                              1313   
10200235 2018-10-31   pierce                              1405   
27843780 2021-12-31  whatcom                                 4   
7409854  2021-04-30    clark                                 0   

          Plug-In Hybrid Electric Vehicles (PHEVs)  \
13240033                                         1   
5174968                                        868   
10200235                                      1305   
27843780                                         0   
7409854                                          0   

          Electric Vehicle (EV) Total  Total Vehicles  \
13240033                            8          182787   
5174968                          2181          314191   
10200235            

In [17]:
import pandas as pd

# Load the sampled data if not already in memory
sampled_data_path = base_path + "sampled_ev_data_10000.parquet"
sampled_data = pd.read_parquet(sampled_data_path)

# Display basic information
print("Data Types:\n", sampled_data.dtypes)
print("\nData Summary:\n", sampled_data.describe(include='all', datetime_is_numeric=True))

# --- Consistency Check ---
print("\n--- Consistency Check ---")

for col in sampled_data.columns:
    print(f"\nColumn: {col}")
    
    # Check for missing values
    missing_count = sampled_data[col].isnull().sum()
    if missing_count > 0:
        print(f"   Missing Values: {missing_count}")
    else:
        print("   No missing values.")

    # Check for mixed data types
    if sampled_data[col].apply(type).nunique() > 1:
        print(f"   Warning: Mixed data types detected in column '{col}'")
    
    # Summary for categorical or string columns
    if sampled_data[col].dtype == 'object':
        unique_values = sampled_data[col].unique()
        print(f"   Sample unique values: {unique_values[:5]}")
    else:
        # For numeric columns, display descriptive statistics
        print(f"   Descriptive Statistics:\n{sampled_data[col].describe()}")

# Check Date column format if present
if 'Date' in sampled_data.columns:
    try:
        sampled_data['Date'] = pd.to_datetime(sampled_data['Date'], errors='raise')
        print("\nDate column is in the correct datetime format.")
    except Exception as e:
        print("\nDate column has issues:", e)

Data Types:
 Date                                              datetime64[ns]
County                                                    object
Battery Electric Vehicles (BEVs)                           int64
Plug-In Hybrid Electric Vehicles (PHEVs)                   int64
Electric Vehicle (EV) Total                                int64
Total Vehicles                                             int64
Percent Electric Vehicles                                float64
Per Capita Personal Income 2020                            int64
Per Capita Personal Income 2021                            int64
Per Capita Personal Income 2022                            int64
Percent Change from preceding period 2020-2021           float64
Percent Change from preceding period 2021-2022           float64
dtype: object

Data Summary:
                               Date County  Battery Electric Vehicles (BEVs)  \
count                        10000  10000                      10000.000000   
unique             

/var/folders/hf/8nj1_hmj5k3fl8yk6q3cl7sc0000gn/T/ipykernel_46254/1892825327.py:34: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(f"   Descriptive Statistics:\n{sampled_data[col].describe()}")


In [18]:
import pandas as pd

# Expected list of Washington counties for verification
wa_counties = [
    "adams", "asotin", "benton", "chelan", "clallam", "clark", "columbia", "cowlitz", "douglas",
    "ferry", "franklin", "garfield", "grant", "grays harbor", "island", "jefferson", "king", "kitsap",
    "kittitas", "klickitat", "lewis", "lincoln", "mason", "okanogan", "pacific", "pend oreille", 
    "pierce", "san juan", "skagit", "skamania", "snohomish", "spokane", "stevens", "thurston", 
    "wahkiakum", "walla walla", "whatcom", "whitman", "yakima"
]

# Check unique values in County column
unique_counties = sampled_data['County'].unique()
print("Unique Counties in Data:", unique_counties)

# Compare with expected counties
unexpected_counties = [county for county in unique_counties if county not in wa_counties]
if unexpected_counties:
    print("Unexpected counties found:", unexpected_counties)
else:
    print("All counties match expected values.")

Unique Counties in Data: ['king' 'clark' 'pierce' 'whatcom' 'snohomish' 'kitsap' 'clallam'
 'franklin' 'chelan' 'san juan' 'skagit' 'thurston' 'stevens' 'grant'
 'spokane' 'cowlitz' 'benton' 'lewis' 'jefferson' 'island' 'mason'
 'okanogan' 'kittitas' 'grays harbor' 'whitman' 'walla walla' 'douglas'
 'yakima' 'pend oreille' 'pacific' 'klickitat' 'adams' 'lincoln'
 'skamania' 'asotin' 'wahkiakum' 'columbia' 'ferry']
All counties match expected values.


In [19]:
# Identify potential outliers by reviewing high percentiles
outlier_columns = [
    'Battery Electric Vehicles (BEVs)', 
    'Plug-In Hybrid Electric Vehicles (PHEVs)', 
    'Electric Vehicle (EV) Total'
]

# Display percentiles for each column to spot outliers
for col in outlier_columns:
    print(f"\n{col} - High Percentile Values:")
    print(sampled_data[col].describe(percentiles=[.5, .75, .9, .95, .99, .999]))


Battery Electric Vehicles (BEVs) - High Percentile Values:
count    10000.000000
mean      9359.138600
std      17537.931144
min          0.000000
50%        621.000000
75%      10937.000000
90%      32269.000000
95%      50962.000000
99%      78656.000000
99.9%    83031.000000
max      83031.000000
Name: Battery Electric Vehicles (BEVs), dtype: float64

Plug-In Hybrid Electric Vehicles (PHEVs) - High Percentile Values:
count    10000.000000
mean      2771.774000
std       4584.411154
min          0.000000
50%         45.000000
75%       4101.000000
90%       9549.000000
95%      12795.000000
99%      19175.000000
99.9%    20025.000000
max      20025.000000
Name: Plug-In Hybrid Electric Vehicles (PHEVs), dtype: float64

Electric Vehicle (EV) Total - High Percentile Values:
count     10000.000000
mean      12130.912600
std       22073.847974
min           0.000000
50%         803.500000
75%       14995.000000
90%       41818.000000
95%       63757.000000
99%       97831.000000
99.9%   

In [20]:
# Check for logical values in Percent Electric Vehicles and compare with Total Vehicles
print("Summary of Percent Electric Vehicles:")
print(sampled_data['Percent Electric Vehicles'].describe())

# Cross-check Percent Electric Vehicles against Total Vehicles to ensure reasonable proportions
sampled_data['EV Percent Consistency'] = (sampled_data['Electric Vehicle (EV) Total'] / sampled_data['Total Vehicles']) * 100
print("\nSample Consistency Check for EV Percentage:")
print(sampled_data[['Percent Electric Vehicles', 'EV Percent Consistency']].head(10))

Summary of Percent Electric Vehicles:
count    10000.000000
mean         1.134465
std          1.592448
min          0.000000
25%          0.000000
50%          0.470000
75%          1.692500
max          7.500000
Name: Percent Electric Vehicles, dtype: float64

Sample Consistency Check for EV Percentage:
   Percent Electric Vehicles  EV Percent Consistency
0                       0.00                0.004377
1                       0.69                0.694164
2                       0.50                0.502171
3                       0.01                0.009162
4                       0.00                0.000000
5                       5.56                5.563035
6                       7.35                7.348826
7                       0.00                0.000821
8                       0.00                0.000825
9                       1.04                1.038733


In [21]:
# Export as Parquet
sampled_data.to_parquet(base_path + "EV_Data_Comprehensive.parquet", index=False)
print("Exported to Parquet format as EV_Data_Comprehensive.parquet.")

# Export as CSV for broader compatibility (optional)
sampled_data.to_csv(base_path + "EV_Data_Comprehensive.csv", index=False)
print("Exported to CSV format as EV_Data_Comprehensive.csv.")

Exported to Parquet format as EV_Data_Comprehensive.parquet.
Exported to CSV format as EV_Data_Comprehensive.csv.
